<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2023notebooks/2023_0215RAM_from_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. P2P の簡単なデモ

<!-- 注意: Colab 上で実行する場合，Google Drive への接続許可を求めるポップアップウィンドウが開くので，許可する必要があります。 -->

In [ ]:
%config InlineBackend.figure_format = 'retina'
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

from IPython import get_ipython
isColab =  'google.colab' in str(get_ipython())

import math
import random
import numpy as np
import time
import gzip
import json
import sys
import re
import json

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

try:
    import jaconv
except ImportError:
    !pip install jaconv
    import jaconv
    

if isColab:

    # termcolor を downgrade しないと colab ではテキストに色がつかない
    !pip install --upgrade termcolor==1.1
    import termcolor    

    # 結果を保存するために Google Drive をマウントする
    # import google.colab
    # google.colab.drive.mount('/content/drive/')
    
    # GPU 情報を表示
    !nvidia-smi -L

    #!pip install ipynbname --upgrade > /dev/null

    !pip install japanize_matplotlib

if isColab:
    # colab 上で MeCab を動作させるために，C コンパイラを起動して，MeCab の構築を行う
    # そのため時間がかかる。
    !apt install aptitude
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
    !pip install mecab-python3==0.7
    !pip install jaconv
    
    import MeCab
    mecab_wakati = MeCab.Tagger('-Owakati').parse
    mecab_yomi = MeCab.Tagger('-Oyomi').parse
    
else:
    from ccap.mecab_settings import yomi as mecab_yomi
    from ccap.mecab_settings import wakati as mecab_wakati


# ここから下は，コード実行に関するバージョン情報などの情報源の取得と表示
from termcolor import colored

import platform
HOSTNAME = platform.node().split('.')[0]

import os
HOME = os.environ['HOME']

try:
    import ipynbname
except ImportError:
    !pip install ipynbname
    import ipynbname
FILEPATH = str(ipynbname.path()).replace(HOME+'/','')

import pwd
USER=pwd.getpwuid(os.geteuid())[0]

from datetime import date
TODAY=date.today()

import torch
TORCH_VERSION = torch.__version__

color = 'green'
print('日付:',colored(f'{TODAY}', color=color, attrs=['bold']))
print('HOSTNAME:',colored(f'{HOSTNAME}', color=color, attrs=['bold']))
print('ユーザ名:',colored(f'{USER}', color=color, attrs=['bold']))
print('HOME:',colored(f'{HOME}', color=color,attrs=['bold']))
print('ファイル名:',colored(f'{FILEPATH}', color=color, attrs=['bold']))
print('torch.__version__:',colored(f'{TORCH_VERSION}', color=color, attrs=['bold']))

In [ ]:
if isColab:
    !git clone https://github.com/ShinAsakawa/RAM.git 

In [23]:
# シミュレーションに必要なパラメータの設定ユーティリティ
from RAM import set_params_from_file
from RAM import set_params_from_config

# json file から読み込むか
json_fname = 'params_test2.json'

# そうでなければ，以下のパラメータを設定して実行
configs = {
    'dataset_name'  : 'vdrj',   # ['pyslex71', 'vdrj', 'onechar', 'fushimi1999']
    #'dataset_name'  : 'fushimi1999',   # ['pyslex71', 'vdrj', 'onechar', 'fushimi1999']
    #'dataset_name'   : 'onechar',
    'traindata_size':  10000,    # 訓練データ (語彙) 数，
    #'traindata_size':  2000,   # 訓練データ (語彙) 数，
    'traindata_ratio': 0.9,     # 訓練データと検証データを分割する比率。ただし onechar データセットでは無効
    #'traindata_ratio': 1.0,     # 訓練データと検証データを分割する比率。ただし onechar データセットでは無効
    #'stop_list': fushimi1999_wordlist,
    'stop_list': None,
    'epochs': 5,               # 学習のためのエポック数
    
    # 以下 `source` と `rget` を定義することで，別の課題を実行可能
    'source': 'phon',          # ['orth', 'phon']
    'target': 'phon',          # ['orth', 'phon']

    #'hidden_size': 256,        # 中間層のニューロン数
    #'hidden_size': 128,
    'hidden_size': 64,

    #'lr' : 0.0001,
    #'lr': 1e-4,                       # 学習率
    'lr': 1e-3,                       # 学習率
    'dropout_p': 0.0,                 # ドロップアウト率
    'teacher_forcing_ratio': 0.5,     # 教師強制を行う確率
    'optim_func': "torch.optim.Adam",   # 最適化アルゴリズム ['torch.optim.Adam', 'torch.optim.SGD', 'torch.optim.AdamW']
    'loss_func' :"torch.nn.NLLLoss",  # 負の対数尤度損失 ['torch.nn.NLLLoss()', or 'torch.nn.CrossEntropyLoss()']
    #'loss_func' :torch.nn.NLLLoss(),

    'random_seed': 42,          # 乱数の種。ダグラス・アダムス著「銀河ヒッチハイカーズガイド」
    'pretrained': False,       # True であれば訓練済ファイルを読み込む
    #'isTrain'   : True,       # True であれば学習する
    
    'verbose'   : False,
    # 学習済のモデルパラメータを保存するファイル名
    #'path_saved': '2022_0607lam_o2p_hid32_vocab10k.pt', 
    #'path_saved': '2023_0220vdrj_2k_p2p.pt',
    'path_saved': False,                      # 保存しない場合
}


#params, encoder, decoder, ds, train_dataset, val_dataset, \
#encoder_optimizer, decoder_optimizer, N_train, N_val = set_params_from_file(
#     json_fname=json_fname, device=device)
# params, encoder, decoder, ds, train_dataset, val_dataset, \
# encoder_optimizer, decoder_optimizer, N_train, N_val = set_params_from_file(params=configs, device=device)

X = set_params_from_config(configs=configs, device=device)
params = X['params']
encoder = X['encoder']
decoder = X['decoder']
ds = X['dataset']
train_dataset = X['train_dataset']
#val_dataset = {'val':X['val_dataset']}
val_dataset = X['val_dataset']
encoder_optimizer = X['encoder_optimizer']
decoder_optimizer = X['decoder_optimizer']
N_train = X['N_train']
N_val   = X['N_val']

configs['max_length'] = ds.maxlen
configs['source_vocab'], configs['target_vocab'] = ds.source_list, ds.target_list
configs['encoder'], configs['decoder'] = encoder, decoder
configs['encoder_optimizer'], configs['decoder_optimizer'] = encoder_optimizer, decoder_optimizer
configs['N_train'], configs['N_val'] = N_train, N_val
configs['train_dataset'], configs['val_dataset'] = train_dataset, val_dataset
configs['device'] = device

In [ ]:
%reload_ext autoreload
%autoreload 2
from RAM import train_epochs_with_config

perfs = train_epochs_with_config(configs=configs, verbose=True)

In [ ]:
plt.plot(perfs['losses'])
plt.show()

#plt.yticks(np.arange(0, 1, step=0.2))
plt.xticks(np.arange(0, len(perfs['train_accuracy']), step=1))
plt.plot(perfs['train_accuracy'], label='訓練データ')
plt.plot(perfs['val_accuracy'], label='検証データ')
plt.legend(loc="upper left")
#plt.ylim(0,1.75)
plt.show()

In [ ]:
from RAM import eval_input_seq2seq
_ = eval_input_seq2seq(encoder=encoder, decoder=decoder, ds=ds)

In [ ]:
from RAM import SALA_Dataset
sala_r31 = SALA_Dataset(task="sala_r29")
sala_r31_words = [v['orth'] for k, v in sala_r31.data_dict.items()]
for wrd in sala_r31_words:
    _phon = jaconv.hiragana2julius(wrd).split()
    print(wrd, eval_input_seq2seq(encoder=encoder, decoder=decoder, ds=ds, inp_wrd=_phon, isPrint=False))